In [1]:
import time
import pandas as pd
import os

In [2]:
from selenium import webdriver

In [3]:
# 기사정보를 저장하는 함수
def article_save(query1, query2, query3, ds, de):
    # 기사제목, 링크주소, 기사내용, 좋아요, 화나요를 저장할 딕셔너리
    article_dict = { 'title':[], 'link':[], 'contents':[], 'like':[], 'angry':[] }
    # cnt : 페이지 변수
    cnt = 1
    # article_search()에서 입력한 조건으로 기사를 검색한다.
    # 한 페이지당 큰 제목 기사 10개씩
    # 네이버는 4000개까지만 제공
    for start in range(1,3992,10) :
        site = f'https://search.naver.com/search.naver?where=news&query={query1}%26{query2}%26{query3}&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={ds}&de={de}&docid=&nso=so%3Ar%2Cp%3Afrom20190101to20200206%2Ca%3Aall&mynews=0&&start={start}&related=0'
        driver.get(site)
        
        # j : 네이버링크 인덱스변수
        j = 0
        for i in range(0, 10, 1) :
            # 기사 중에 (큰 제목만 탐색)
            # 네이버링크가 있으면 
            # 딕셔너리에 기사제목과 링크주소 추가 후
            # 기사를 열어본다.
            preview_list = driver.find_elements_by_css_selector('ul.type01 > li > dl')
            preview = preview_list[i]
            # 링크 인덱스 접근은 기사제목과 독립적으로 한다.
            # 한 페이지에 네이버링크가 하나도 없는 경우에는 바로 다음페이지로 간다
            try:
                link_list = driver.find_elements_by_css_selector('ul.type01 > li > dl > dd.txt_inline > a')
                link = link_list[j]
            # 한 페이지에 네이버링크가 하나도 없는 경우에는 바로 다음페이지로 간다
            except:
                pass
            
            else:
                if ('네이버뉴스' in preview.text) == True :
                    # 기사 제목과 링크 딕셔너리에 추가
                    title = preview.find_element_by_css_selector('dt')
                    article_dict['title'].append(title.text.strip(''))
                    article_dict['link'].append(link.get_attribute('href'))
                    link.click()
                    # 링크 인덱스가 마지막이 아니면 다음 링크 접근
                    if j < (len(link_list)-1) :
                        j += 1
                    # 마지막이라면 인덱스 변수 증가 안함
                    else :
                        pass

                    # 네이버뉴스가 열린 탭을 탐색한다.
                    news_tab = driver.window_handles[-1]
                    driver.switch_to.window(window_name=news_tab)

                    # 연예뉴스는 태그명이 다름
                    if 'entertain' in driver.current_url:
                        try:
                            # 좋아요 수와 화나요 수를 탐색한다.
                            like = driver.find_element_by_css_selector('#ends_addition > div._reactionModule.u_likeit > ul > li.u_likeit_list.good > a > span.u_likeit_list_count._count')
                            angry = driver.find_element_by_css_selector('#ends_addition > div._reactionModule.u_likeit > ul > li.u_likeit_list.angry > a > span.u_likeit_list_count._count')

                        except:
                            # 좋아요와 화나요가 없으면 0으로 추가
                            contents = driver.find_element_by_css_selector('#articeBody')
                            article_dict['contents'].append(contents.text.strip(''))

                            article_dict['like'].append('0')
                            article_dict['angry'].append('0')


                        else:
                            # 딕셔너리에 기사내용, 좋아요 수, 화나요 수를 추가한다.(1000이상이면 콤마 제거)
                            contents = driver.find_element_by_css_selector('#articeBody')
                            article_dict['contents'].append(contents.text.strip(''))

                            article_dict['like'].append(like.text.replace('[\,]', ''))
                            article_dict['angry'].append(angry.text.replace('[\,]', ''))
                        # 공감 버튼 유무와 관계없이 실행
                        finally:
                            # 기사 탭을 닫고 검색결과 화면으로 돌아간다.
                            driver.close()
                            first_tab = driver.window_handles[0]
                            driver.switch_to.window(window_name=first_tab)

                    else:
                        try:
                            # 좋아요 수와 화나요 수를 탐색한다.
                            like = driver.find_element_by_css_selector('#spiLayer > div._reactionModule.u_likeit > ul > li.u_likeit_list.good > a > span.u_likeit_list_count._count')
                            angry = driver.find_element_by_css_selector('#spiLayer > div._reactionModule.u_likeit > ul > li.u_likeit_list.angry > a > span.u_likeit_list_count._count')

                        except:
                            # 좋아요와 화나요가 없으면 0으로 추가
                            contents = driver.find_element_by_css_selector('#articleBodyContents')
                            article_dict['contents'].append(contents.text.strip(''))

                            article_dict['like'].append('0')
                            article_dict['angry'].append('0')


                        else:
                            # 딕셔너리에 기사내용, 좋아요 수, 화나요 수를 추가한다.(1000이상이면 콤마 제거)
                            contents = driver.find_element_by_css_selector('#articleBodyContents')
                            article_dict['contents'].append(contents.text.strip(''))

                            article_dict['like'].append(like.text.replace('[\,]', ''))
                            article_dict['angry'].append(angry.text.replace('[\,]', ''))
                        # 공감 버튼 유무와 관계없이 실행
                        finally:
                            # 기사 탭을 닫고 검색결과 화면으로 돌아간다.
                            driver.close()
                            first_tab = driver.window_handles[0]
                            driver.switch_to.window(window_name=first_tab)
                    
        # 내부for문 종료
    
        # 데이터프레임으로 추가
        df = pd.DataFrame.from_dict(article_dict)
        df.to_csv('naver_article.csv', encoding='utf-8-sig', mode='w', index=False)
        print(f'{cnt}페이지 탐색 완료')
        cnt += 1
        
    # 외부 for문 종료
    return print('\n수집완료')

In [4]:
# 네이버뉴스를 검색하는 함수
def article_search():
    # 검색어, 기간을 입력한다.
    # 날짜 형식 : yyyy.mm.dd
    query1 = input('키워드1 : ')
    query2 = input('키워드2 : ')
    query3 = input('키워드3 : ')
    ds = input('시작일 : ')
    de = input('종료일 : ')
    
    # 기사 저장하는 함수 호출
    article_save(query1, query2, query3, ds, de)

In [5]:
driver = webdriver.Chrome()

article_search()

키워드1 : 서울
키워드2 : 집값상승
키워드3 : 
시작일 : 2016.01.01
종료일 : 2019.12.31
1페이지 탐색 완료
2페이지 탐색 완료
3페이지 탐색 완료
4페이지 탐색 완료
5페이지 탐색 완료
6페이지 탐색 완료
7페이지 탐색 완료
8페이지 탐색 완료
9페이지 탐색 완료
10페이지 탐색 완료
11페이지 탐색 완료
12페이지 탐색 완료
13페이지 탐색 완료
14페이지 탐색 완료
15페이지 탐색 완료
16페이지 탐색 완료
17페이지 탐색 완료
18페이지 탐색 완료
19페이지 탐색 완료
20페이지 탐색 완료
21페이지 탐색 완료
22페이지 탐색 완료
23페이지 탐색 완료
24페이지 탐색 완료
25페이지 탐색 완료
26페이지 탐색 완료
27페이지 탐색 완료
28페이지 탐색 완료
29페이지 탐색 완료
30페이지 탐색 완료
31페이지 탐색 완료
32페이지 탐색 완료
33페이지 탐색 완료
34페이지 탐색 완료
35페이지 탐색 완료
36페이지 탐색 완료
37페이지 탐색 완료
38페이지 탐색 완료
39페이지 탐색 완료
40페이지 탐색 완료
41페이지 탐색 완료
42페이지 탐색 완료
43페이지 탐색 완료
44페이지 탐색 완료
45페이지 탐색 완료
46페이지 탐색 완료
47페이지 탐색 완료
48페이지 탐색 완료
49페이지 탐색 완료
50페이지 탐색 완료
51페이지 탐색 완료
52페이지 탐색 완료
53페이지 탐색 완료
54페이지 탐색 완료
55페이지 탐색 완료
56페이지 탐색 완료
57페이지 탐색 완료
58페이지 탐색 완료
59페이지 탐색 완료
60페이지 탐색 완료
61페이지 탐색 완료
62페이지 탐색 완료
63페이지 탐색 완료
64페이지 탐색 완료
65페이지 탐색 완료
66페이지 탐색 완료
67페이지 탐색 완료
68페이지 탐색 완료
69페이지 탐색 완료
70페이지 탐색 완료
71페이지 탐색 완료
72페이지 탐색 완료
73페이지 탐색 완료
74페이지 탐색 완료
75페이지 탐색 완료
76페이지 탐색 완료
77페이지 탐색 완료
78페이지 탐색 완료
79페이지 탐색 